<a href="https://colab.research.google.com/github/15H44N/Book-Indexing/blob/master/BookindexingLookup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and Configure


In [0]:
!pip install isbnlib

In [0]:
import pandas as pd
import isbnlib as ib
from isbnlib import ISBNLibException
from bs4 import BeautifulSoup as soup
import requests
import numpy as np
import json
import textwrap
import urllib.request

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

# Data Import

In [0]:
worksheet = gc.open('Books Data').sheet1

# get_all_values gives a list of rows.
allrows = worksheet.get_all_values()

labels=allrows.pop(0)
rowdict={}
rowdictkeys=[]
for i in labels:
  rowdictkeys.append(i)
allrowsT=np.transpose(allrows)
#print(allrowsT)
rowdictvalues=allrowsT
rowdict=dict(zip(rowdictkeys,rowdictvalues))
sdf=pd.DataFrame(rowdict)
sdf.head()

In [0]:
Isbnlist=sdf['LV Shelf Left'].tolist()

# Lookup Functions

In [0]:
def getBookValsGoogleBooks(isbn="9788129300379"):

  base_api_link = "https://www.googleapis.com/books/v1/volumes?q=isbn:"

  savelist=["publishedDate","title","authors","pageCount","categories","infoLink"]
  namedik={'Author': 'Swami Ramsukhdas','Categories': 'Conduct of life','ISBN-13': '9788129300379','Published': '2010','Title': 'Be Good and Invaluable Advice','URL': 'http://books.google.com/books?id=zOUmMwEACAAJ&dq=isbn:9788129300379&hl=&source=gbs_api'}

  with urllib.request.urlopen(base_api_link + isbn) as f:
    text = f.read()
    decoded_text = text.decode("utf-8")
    obj = json.loads(decoded_text) 

    if obj["totalItems"] <= 0:
      errdict={}
      for i in namedik.keys():
        errdict[i]=""
      return errdict
        
    
    volume_info = obj["items"][0] 
    for i in savelist:
      if i not in volume_info['volumeInfo'].keys():
        volume_info['volumeInfo'][i]=''

    authors = obj["items"][0]["volumeInfo"]["authors"]

  df=pd.DataFrame.from_dict(volume_info)
  df.drop(columns=['kind','id','etag','selfLink','saleInfo','accessInfo'],inplace=True)
  df=df.transpose()
  labeltodroplist=[]

  authlist=df['authors'][0]
  df['authors'][0]=", ".join(authlist)

  if df['categories'] is not '':
    catlist=df['categories'][0]
    df['categories'][0]=", ".join(catlist)
  
  for i in range(len(df.transpose())):
    if df.columns[i] not in savelist:
      labeltodroplist.append(df.columns[i])
  df.drop(inplace=True,axis='columns',labels=labeltodroplist)
  df.rename(columns={"authors":"Author","categories":"Categories","publishedDate":"Published","title":"Title","infoLink":"URL"},inplace=True)
  #df['ISBN-13']=[(isbn)]
  df.drop(inplace=True,columns=["pageCount"])
  print(df.to_dict(orient='records')[0])
  return(df.to_dict(orient='records')[0])


In [0]:
def getpageISBN(isbn="9788129300379"):

  apikey="http://api.scraperapi.com?api_key=6594089066e419a8ba582146468a34af&url="
  apiurl="http://isbnsearch.org/isbn/"

  #r=requests.get(apiurl+isbn)
  #if r is "<Response [403]>":
  req=requests.get(apikey+apiurl+isbn)
  return(req)

def getbookvals(r):
  page_soup=soup(r.content,"html.parser")
  #print(page_soup.prettify())
  booklist=page_soup.findAll("div",{"class":"bookinfo"})
  if len(booklist)==0:
      return {"ISBN-13":isbn}
  page2=soup(str(booklist[0]),"html.parser")
  title=soup((str(page2.find("h1"))),"html.parser").text
  plist=page2.findAll("p")
  infol=[('Title',title)]
  for i in plist:
    vals=i.text.split(sep=": ",maxsplit=1)
    infol.append((vals[0],vals[1]))
  infodict=dict(infol)
  infodict["URL"]=str("http://isbnsearch.org/isbn/"+infodict["ISBN-13"])
  return(infodict)

# Lookup in Google Books and ISBN Search


In [0]:
infolist=[]
row={}
gRowdict={}
for i in Isbnlist:
  gRowdict=getBookValsGoogleBooks(i)
  gRowdict["ISBN-13"]=i
  infolist.append(gRowdict)

In [0]:
for i in range(len(infolist)):
  y=infolist[i]
  if (y["Author"] is "" and y["Title"] is ""):
    #not found in google books.
    #find in the web scraper method
    isbn=Isbnlist[i]
    rst=getpageISBN(isbn)
    l1=getbookvals(rst)
    infolist[i]=l1

In [0]:
df=pd.DataFrame(infolist)
df.drop(inplace=True,columns=["ISBN-10","Edition","Binding","Publisher"])
df.fillna("Not Available")
df.to_csv("/content/drive/My Drive/Colab Notebooks/Books Arrange/data.csv",index=False)
df

# Lookup on OpenLib

In [0]:
df=pd.read_csv("/content/drive/My Drive/Colab Notebooks/Books Arrange/data.csv",)
df.fillna("")

In [0]:
nfdict={}
nflist=[]
for i in range(len(df['Title'])):
    if (pd.isna(df['Title'][i])):
        nfdict["index"]=i
        print(i,df["ISBN-13"][i])
        nfdict["isbn"]=df["ISBN-13"][i]
        nflist.append(nfdict)
        nfdict={}

In [0]:
nfinfolist=[]
infodict={}

notFound=False
namesofmeta=['Authors','Language','Publisher','Title','Year']


for i in nflist:
    isbnval=list((i.values()))[1]
    print((isbnval))
    infodict={}
    exfodict={}
   
    try:
        infodict= ib.meta(str(isbnval),service="openl")
    except ISBNLibException:
        infodict = {"ISBN-13":str(isbnval)}
        notFound=True
        for i in namesofmeta:
          infodict[i]=""
      
    if not(False):
      authname=""
      for j in infodict["Authors"]:
        if not isinstance(infodict["Authors"],str):
          authname=", ".join(j)
        else:
          authname=str(infodict["Authors"])
      exfodict["Author"]=authname
      exfodict["Title"]=infodict["Title"]
      exfodict["ISBN-13"]=infodict["ISBN-13"]
      exfodict["Published"]=infodict["Year"]
      exfodict["URL"]=""
      exfodict["Categories"]=""
      #exfodict["Publisher"]=infodict["Publisher"]
    print(exfodict)
    
    nfinfolist.append(exfodict)  

In [0]:
nfupdatelist=[]
for i in nfinfolist:
  row=[]
  row.append(i["Title"])
  row.append(i["Author"])
  row.append(i["Published"])
  row.append(i["Categories"])
  row.append(i["URL"])
  row.append(i["ISBN-13"])
  nfupdatelist.append(row)

j=0
for i in nflist:
  index=list(i.values())[0]
  df.loc[index]=nfupdatelist[j]
  j=j+1

df

# Lookup on Amazon and others

In [0]:
def scrapeamazon(isbn="9788181479136"):
  apihead="http://api.scraperapi.com?api_key=6594089066e419a8ba582146468a34af&url="
  pageurl="https://www.amazon.com/s?k="
  r=requests.get(pageurl+isbn)
  #if (r.status_code>500):
  #  r1=requests.get(apihead+pageurl+isbn)
  #  r=r1
  page_soup=soup(r.content,"html.parser")
  datalist=page_soup.findAll("div",{"class":"sg-col-inner"})
  return(datalist)

#scrapeamazon()

In [0]:
def scrapeisbnsnet(isbn="9788189988364"):
  apihead="http://api.scraperapi.com?api_key=6594089066e419a8ba582146468a34af&url="
  pageurl="https://www.isbns.net/isbn/"
  r=requests.get(apihead+pageurl+isbn+"/")
  print(r.content)
  page_soup=soup(r.content,"html.parser")
  datalist=page_soup.findAll("td",{"class":"regularText"})
  return(datalist)

#datalist=scrapeisbnsnet()

In [0]:
df.to_csv("./output.csv",index=False)